In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

In [2]:
def get_ride_data(csv_file, ride_id, month, day):
    url = f"https://queue-times.com/en-US/parks/16/rides/{ride_id}?given_date=2025-{month}-{day}"
    page = requests.get(url)
    doc = BeautifulSoup(page.content, "lxml")
    #scrape the data
    try:
        front_index = doc.find_all('script')[7].text.index("2ecc71")+12
        end_index = doc.find_all('script')[7].text.index("Reported closed")-14
        script = doc.find_all('script')[7].text.strip()[front_index:end_index]
        data = json.loads(script)#convert to object
        
        df1 = pd.DataFrame(data, columns=['Date', 'Wait'])
        df1['Date'] = pd.to_datetime(df1['Date'], format="%m/%d/%y %H:%M:%S").dt.floor("min")
        df1['Wait'] = df1['Wait'].astype(float)
        df1['Wait'] = df1['Wait'].astype(int)
        df1.to_csv(csv_file, mode='a', index=False, header=False)
        print(day, end=" ")
    except (IndexError, ValueError, json.JSONDecodeError) as e:
        print(f"{day} not found")

In [3]:
csv_file = "spacemountain.csv"
ride_id = input("Ride ID:")
months = input("Months:").split()
df = pd.DataFrame(columns=['Date', 'Wait'])
df.to_csv(csv_file, mode='w', index=False, header=True)

for month in months:
        for i in range(30):
            get_ride_data(csv_file, ride_id, month, i+1)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

In [4]:
df = pd.read_csv(csv_file)
df.index = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S')
wait = df['Wait']
wait.plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.